In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

In [ ]:
data2 = pd.read_csv('./data.csv')
data2

In [ ]:
data2[data2.columns] = data2[data2.columns].astype('Int64')

data2 = data2.drop('Final_EDSS', axis=1)
data2 = data2.drop('Unnamed: 0', axis=1)
data2 = data2.drop('Initial_EDSS', axis=1)

data2['group'] = data2['group'].replace(2, 0)
data2

In [ ]:
data2["group"  ].value_counts()

In [ ]:
# Verificando si hay valores nulos por columna
print(data2.isnull().sum())

In [ ]:
from sklearn.impute import SimpleImputer
# Crear un imputador para las columnas con valores faltantes
imputer = SimpleImputer(strategy='mean')

# Seleccionar las columnas con valores faltantes
columns_with_nan = ['Schooling', 'Initial_Symptom']

# Imputar los valores faltantes en las columnas específicas
data2[columns_with_nan] = imputer.fit_transform(data2[columns_with_nan])

# Verificar si aún hay valores nulos en el DataFrame
if data2.isnull().values.any():
    # Imputar cualquier otro valor faltante en el DataFrame completo
    data2 = pd.DataFrame(imputer.fit_transform(data2), columns=data2.columns)


X = data2.drop('group', axis=1)
y = data2['group']

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
# Matriz de confusión
conf_matrix = confusion_matrix(y_val, y_pred_val)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
# Calcular AUC-ROC
y_prob_val = model.predict_proba(X_val)[:, 1]
auc_roc = roc_auc_score(y_val, y_prob_val)
print("AUC-ROC:", auc_roc)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_val, y_prob_val)
plt.plot(fpr, tpr,)
plt.plot([0, 1], [0, 1], 'k--')
plt.title('ROC Curve')
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Curva ROC (area = %0.2f)' % auc_roc)
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

#entrenamiento
y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)


#validacion
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)

# prueba
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)



print(f'Precisión en el conjunto de entrenamiento: {train_accuracy}')
print(f'Precisión en el conjunto de validación: {val_accuracy}')
print(f'Precisión en el conjunto de prueba: {test_accuracy}')

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import classification_report


# Crear un selector secuencial
sfs = SequentialFeatureSelector(model, n_features_to_select=5, direction='forward', cv=5)

# Ajustar el selector a los datos de entrenamiento
sfs.fit(X_train, y_train)

# Obtener las características seleccionadas
selected_features = X_train.columns[sfs.get_support()]

# Crear un nuevo DataFrame con las características seleccionadas
X_train_selected = X_train[selected_features]
X_val_selected = X_val[selected_features]



# Evaluar el modelo en el conjunto de prueba
y_predd = model.predict(X_test)

# Calcular métricas
accuracy = accuracy_score(y_test, y_predd)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_predd))
print("AUC-ROC:", roc_auc_score(y_test, y_predd))

In [ ]:
# Guardar las características seleccionadas
selected_features = X_train.columns[sfs.get_support()]

# Entrenar el modelo con las características seleccionadas en el conjunto de entrenamiento completo
model.fit(X_train[selected_features], y_train)

# Evaluar el modelo en el conjunto de validación y prueba
y_val_pred_selected = model.predict(X_val[selected_features])
y_test_pred_selected = model.predict(X_test[selected_features])

# Calcular y mostrar métricas para validación y prueba
print("Validación con características seleccionadas:")
print(classification_report(y_val, y_val_pred_selected))
print("AUC-ROC:", roc_auc_score(y_val, model.predict_proba(X_val[selected_features])[:, 1]))

print("Prueba con características seleccionadas:")
print(classification_report(y_test, y_test_pred_selected))
print("AUC-ROC:", roc_auc_score(y_test, model.predict_proba(X_test[selected_features])[:, 1]))

In [ ]:
# Modelo con regularización L1
model_l1 = LogisticRegression(penalty='l1', solver='liblinear')
model_l1.fit(X_train, y_train)

# Predicciones y evaluación
y_pred_l1 = model_l1.predict(X_test)
print("Resultados con regularización L1:")
print(classification_report(y_test, y_pred_l1))
print("AUC-ROC:", roc_auc_score(y_test, model_l1.predict_proba(X_test)[:, 1]))

In [ ]:
# Modelo con regularización L2  --- ESTE ES MEJOR
model_l2 = LogisticRegression(penalty='l2')
model_l2.fit(X_train, y_train)

y_pred_l2 = model_l2.predict(X_test)
print("Resultados con regularización L2:")
print(classification_report(y_test, y_pred_l2))
print("AUC-ROC:", roc_auc_score(y_test, model_l2.predict_proba(X_test)[:, 1]))

In [ ]:
import itertools

def evaluate_model(X, y):
    # Divide los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrena el modelo
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Evalúa el modelo
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Columnas que siempre se eliminan
columns_to_remove = ['Final_EDSS', 'Unnamed: 0', 'Initial_EDSS', 'group']

# Obtener las columnas restantes (características)
features = [col for col in data2.columns if col not in columns_to_remove]

# Generar todas las posibles combinaciones de características, iniciando con todas las características y eliminando una por una
all_combinations = []
for i in range(len(features)):
    combinations = itertools.combinations(features, len(features) - i)
    all_combinations.extend(combinations)

best_accuracy = 0
best_combination = None

for combo in all_combinations:
    # Crear un nuevo DataFrame con las columnas seleccionadas
    X_subset = data2[list(combo)]
    y = data2['group']  # Asegurarse de seleccionar siempre la columna 'group' como target

    # Evalúa el modelo
    accuracy = evaluate_model(X_subset, y)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_combination = combo

print("Mejor combinación de columnas:", best_combination)
print("Mejor precisión:", best_accuracy)